In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [5]:
df_raw.shape

(37224, 14)

In [10]:
df_an = df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [11]:
df_an.shape

(6000, 14)

In [12]:
df_an = RSI(df_an)

In [15]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
5995,2021-12-30 19:00:00+00:00,721,155.312,155.444,155.274,155.444,155.296,155.432,155.260,155.432,155.328,155.458,155.289,155.457,61.286263
5996,2021-12-30 20:00:00+00:00,1001,155.442,155.470,155.289,155.312,155.429,155.458,155.275,155.298,155.456,155.484,155.303,155.326,55.345932
5997,2021-12-30 21:00:00+00:00,638,155.317,155.382,155.286,155.382,155.301,155.362,155.270,155.362,155.333,155.404,155.300,155.402,57.688104
5998,2021-12-30 22:00:00+00:00,623,155.353,155.381,155.282,155.329,155.278,155.315,155.207,155.254,155.428,155.456,155.357,155.404,55.322082
5999,2021-12-30 23:00:00+00:00,419,155.336,155.360,155.284,155.310,155.262,155.329,155.262,155.294,155.411,155.418,155.299,155.327,54.459758
